<a href="https://colab.research.google.com/github/jchen8000/DemystifyingLLMs/blob/main/6_Deployment/LangChain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 6. Deployment of LLMs

## 6.10 LangChain

### **Building a Simple Pipeline with LangChain and HuggingFace Endpoints**

This notebook demonstrates how to build a text classification pipeline using **LangChain** and the **HuggingFace Inference API**. Specifically, we use the a model in HuggingFace to categorize customer reviews into **Positive**, **Negative**, or **Neutral** sentiments.

---

### **Key Objectives**
* **Prompt Templating**: Using LangChain's `PromptTemplate` to standardize instructions for the LLM.
* **LLM Integration**: Connecting to Hugging Face's hosted inference endpoints without needing local GPU resources.
* **Batch Processing**: Efficiently running multiple inputs through a chain simultaneously using the `.batch()` method.

---

###⚠️ **Compatibility Note**

This notebook is confirmed working as of **January 2026**, including the currently available ```google/gemma-2-9b-it``` model. However, LangChain, Hugging Face APIs, and hosted models evolve rapidly, and future updates or model deprecations may require adjustments to this notebook.

**This example is for learning and reference purposes and reflects the current best practices, but the APIs used here may change over time.**

---

In [ ]:
%pip install -q \
  langchain==1.2.3 \
  langchain-core==1.2.6 \
  langchain-huggingface==1.2.0

In [2]:
import os
from google.colab import userdata
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

### Prompt template

In [3]:
# Define a simple prompt template
template = """
The customer review is: {review}
Please classify it as Positive, Negative, or Neutral. Reply with the classification only.
"""
prompt = PromptTemplate.from_template(template)

review_text = "The shipping was quick and the item was perfect. Totally satisfied!"

prompt_text = prompt.format(review=review_text)
print(f"Prompt:\n{prompt_text}")


Prompt:

The customer review is: The shipping was quick and the item was perfect. Totally satisfied!
Please classify it as Positive, Negative, or Neutral. Reply with the classification only.



### Access a LLM model on HuggingFace hub

### Inference with a single prompt:

In [ ]:
os.environ["HUGGINGFACEHUB_API_TOKEN"] = userdata.get('HF_TOKEN')

llm = HuggingFaceEndpoint(
    repo_id="google/gemma-2-9b-it",
    max_new_tokens=20,
    temperature=0.1
)

chat_model = ChatHuggingFace(llm=llm)

chain = prompt | chat_model | StrOutputParser()

response = chain.invoke({"review": review_text})

print(f"Review: {review_text}")
print(response)

Review text: The shipping was quick and the item was perfect. Totally satisfied!
Positive 



### Inference with multiple prompts:

In [5]:
reviews = [
    {'review': "The shipping was quick and the item was perfect. Totally satisfied!"},
    {'review': "The restaurant was terrible, and the service was even worse. Not going back there again."},
    {'review': "The new update is almost useless, the new features seem not relevant to our business flow."},
]
results = chain.batch(reviews)

print("--- Sentiment Analysis Results ---")
for original, classification in zip(reviews, results):
    print(f"Review: {original['review']}")
    print(f"Sentiment: {classification.strip()}\n")

--- Sentiment Analysis Results ---
Review: The shipping was quick and the item was perfect. Totally satisfied!
Sentiment: Positive

Review: The restaurant was terrible, and the service was even worse. Not going back there again.
Sentiment: Negative

Review: The new update is almost useless, the new features seem not relevant to our business flow.
Sentiment: Negative

